In [ ]:
import cv2
import numpy as np
import dlib
from keras.models import load_model

# Load the saved model
model = load_model('/Users/chittireddyruthwik/model.h5')

# Define the emotions and their corresponding labels
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Load the face landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/chittireddyruthwik/Downloads/shape_predictor_68_face_landmarks.dat")

# Capture and process video frames
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:  # Check if frame is captured successfully
        print("Failed to capture video frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        landmarks = predictor(gray, face)

        # Extract face region
        face_roi = gray[y1:y2, x1:x2]
        face_roi = cv2.resize(face_roi, (48, 48))  # Resize to match input shape of model
        face_roi = np.reshape(face_roi, [1, 48, 48, 1])  # Reshape to match input shape of model
        face_roi = face_roi / 255.0  # Normalize pixel values to [0, 1]

        # Make a prediction using the trained model
        pred = model.predict(face_roi)

        # Get the index of the predicted class label
        pred_idx = np.argmax(pred)

        # Draw predicted emotion label on frame
        cv2.putText(frame, emotions[pred_idx], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow('Emotion Analysis', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

